In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score,recall_score,make_scorer,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
test_df = pd.read_csv("test.csv")

In [4]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### 分析

In [6]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
train_df["Parch"].value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

#### Cabinの列を削除(欠損値が多すぎるため、先に削除)

In [8]:
train_df=train_df.drop("Cabin",axis=1)
test_df=test_df.drop("Cabin",axis=1)

### 欠損値の「補完」

In [9]:
train_df_woNaN=train_df.fillna({"Age":train_df["Age"].mean()}).dropna().reset_index(drop=True)
test_df_woNaN =test_df.fillna({"Age":train_df["Age"].mean()}).dropna().reset_index(drop=True)
# indexの振りなおしをしないと、後でconcatする際、正しく結合されないので注意

In [43]:
train_df_woNaN["Family_num"]=train_df_woNaN["SibSp"] + train_df_woNaN["Parch"]
train_df_woNaN.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Family_num
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0


# 定量データでRF

In [10]:
X = train_df_woNaN.loc[:,["Pclass","Age","SibSp","Parch","Fare"]].values
y = train_df_woNaN.loc[:,"Survived"].values

In [11]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.25)

In [12]:
clf_rf = RandomForestClassifier(n_estimators=1000,max_features=4,random_state=1,n_jobs=-1)

In [13]:
clf_rf.fit(X=X_train,y=y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=4, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [14]:
importances = pd.DataFrame(train_df_woNaN.loc[:,["Pclass","Age","SibSp","Parch","Fare"]].columns,columns=['features_label'])
importances['importances'] =clf_rf.feature_importances_
importances.sort_values('importances',ascending=False)

,features_label,importances
4,Fare,0.413506
1,Age,0.380588
0,Pclass,0.089906
2,SibSp,0.066915
3,Parch,0.049085


In [15]:
y_pred = clf_rf.predict(X_test)
confusion_matrix(y_true=y_test,y_pred=y_pred)

array([[106,  30],
       [ 43,  44]], dtype=int64)

In [16]:
def scores(X_test,y_test):
    y_pred = clf_rf.predict(X_test)
    print("accuracy : ",clf_rf.score(X=X_test,y=y_test))
    print("precision : ",precision_score(y_true=y_test,y_pred=y_pred,pos_label=1))
    print("recall : ",recall_score(y_true=y_test,y_pred=y_pred,pos_label=1))

In [17]:
scores(X_test,y_test)

accuracy :  0.672645739910314
precision :  0.5945945945945946
recall :  0.5057471264367817


# +k分割交差検証

In [18]:
precision_scorer = make_scorer(score_func=precision_score,pos_label=1)
scores_data = cross_val_score(estimator=clf_rf,X=X,y=y,cv=5,n_jobs=-1,scoring="accuracy")

In [19]:
scores_data.mean()

0.6614359169681966

# +特徴を増やす(Cabinは除く)

#####  到着場所と性別のエンコード、Onehotエンコード

In [20]:
le = []
enc_columns = np.array([])
count=0
enc_df=train_df_woNaN.copy()
for i in ["Sex","Embarked"]:
    le = np.append(le,LabelEncoder())
    le[count].fit(enc_df.loc[:,i])
    enc_columns = np.append(enc_columns,le[count].classes_)
    enc_df.loc[:,i] = le[count].transform(enc_df.loc[:,i])
    count+=1

In [21]:
enc_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,2


In [22]:
enc_columns

array(['female', 'male', 'C', 'Q', 'S'], dtype=object)

In [23]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(enc_df.loc[:,["Sex","Embarked"]])
enc_data = one_hot_encoder.transform(enc_df.loc[:,["Sex","Embarked"]]).toarray() # numpyマトリックス型で返されるデータをnumpy.array型に変換
hot_df = pd.DataFrame(enc_data)
hot_df.head()

,0,1,2,3,4
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0


In [24]:
enc_df.loc[:,["Pclass","Age","SibSp","Parch","Fare","Survived"]].shape

(889, 6)

In [25]:
hot_df.columns = enc_columns
merge_df = pd.concat([hot_df,enc_df.loc[:,["Pclass","Age","SibSp","Parch","Fare","Survived"]]],axis=1)

In [26]:
merge_df.head()

,female,male,C,Q,S,Pclass,Age,SibSp,Parch,Fare,Survived
0,0.0,1.0,0.0,0.0,1.0,3,22.0,1,0,7.2500,0
1,1.0,0.0,1.0,0.0,0.0,1,38.0,1,0,71.2833,1
2,1.0,0.0,0.0,0.0,1.0,3,26.0,0,0,7.9250,1
3,1.0,0.0,0.0,0.0,1.0,1,35.0,1,0,53.1000,1
4,0.0,1.0,0.0,0.0,1.0,3,35.0,0,0,8.0500,0


### 標準化

In [27]:
from sklearn.preprocessing import StandardScaler

In [28]:
sc = StandardScaler()
sc.fit(merge_df.iloc[:,:-1])
sc.transform(merge_df.iloc[:,:-1])
sc_df = pd.DataFrame(sc.transform(merge_df.iloc[:,:-1]))
sc_df.head()
sc_df['Survived']=merge_df['Survived']
sc_df.columns=merge_df.columns


In [29]:
sc_df.head()

,female,male,C,Q,S,Pclass,Age,SibSp,Parch,Fare,Survived
0,-0.735342,0.735342,-0.482711,-0.307941,0.616794,0.825209,-0.590495,0.431350,-0.474326,-0.500240,0
1,1.359911,-1.359911,2.071634,-0.307941,-1.621287,-1.572211,0.643971,0.431350,-0.474326,0.788947,1
2,1.359911,-1.359911,-0.482711,-0.307941,0.616794,0.825209,-0.281878,-0.475199,-0.474326,-0.486650,1
3,1.359911,-1.359911,-0.482711,-0.307941,0.616794,-1.572211,0.412509,0.431350,-0.474326,0.422861,1
4,-0.735342,0.735342,-0.482711,-0.307941,0.616794,0.825209,0.412509,-0.475199,-0.474326,-0.484133,0


### 完成

In [30]:
X = sc_df.iloc[:,:-1].values
y = sc_df.loc[:,"Survived"].values

In [31]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.25)

In [32]:
clf_rf = RandomForestClassifier(n_estimators=2000,max_features=4,random_state=1,n_jobs=-1)

In [33]:
clf_rf.fit(X=X_train,y=y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=4, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [36]:
v=pd.DataFrame(merge_df.iloc[:,:-1].columns)
v["power"]=clf_rf.feature_importances_
v.sort_values("power",ascending=False)

,0,power
9,Fare,0.257975
6,Age,0.256922
1,male,0.145575
0,female,0.128181
5,Pclass,0.093656
7,SibSp,0.045756
8,Parch,0.036800
2,C,0.014970
4,S,0.012303
3,Q,0.007862


In [37]:
y_pred = clf_rf.predict(X_test)
confusion_matrix(y_true=y_test,y_pred=y_pred)

array([[104,  17],
       [ 30,  72]], dtype=int64)

In [38]:
def scores(X_test,y_test):
    y_pred = clf_rf.predict(X_test)
    print("accuracy : ",clf_rf.score(X=X_test,y=y_test))
    print("precision : ",precision_score(y_true=y_test,y_pred=y_pred,pos_label=1))
    print("recall : ",recall_score(y_true=y_test,y_pred=y_pred,pos_label=1))

In [39]:
scores(X_test,y_test)

accuracy :  0.7892376681614349
precision :  0.8089887640449438
recall :  0.7058823529411765


In [40]:
clf_rf = RandomForestClassifier(n_estimators=1000,max_features=4,random_state=1,n_jobs=-1)
precision_scorer = make_scorer(score_func=precision_score,pos_label=1)
scores_data = cross_val_score(estimator=clf_rf,X=X,y=y,cv=5,n_jobs=-1,scoring="accuracy")

In [41]:
scores_data.mean()

0.8144353456484479

# merge_dfのデータ分析

In [ ]:
merge_df.head()

In [ ]:
is_survived_df = merge_df[merge_df["Survived"] ==1]
isnot_survived_df = merge_df[merge_df["Survived"] ==0]

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(1,1,1)
is_survived_df.plot(y="Age",bins=12,kind="hist",ax=ax,range=[0, 100])
isnot_survived_df.plot(y="Age",bins=12,kind="hist",ax=ax,range=[0, 100],alpha=0.5)
plt.legend(["survived","not_survived"])

### 兄弟or配偶者数毎

In [ ]:
a=merge_df.pivot_table("C",aggfunc="count",index="SibSp",columns="Survived")
a.plot(kind="bar",stacked=True)

In [ ]:
a=a.div(a.sum(axis=1),axis=0)
a.plot(kind="bar",stacked=True)

### 親or子供数毎

In [ ]:
a=merge_df.pivot_table("C",aggfunc="count",index="Parch",columns="Survived")
a.plot(kind="bar",stacked=True)

In [ ]:
a=a.div(a.sum(axis=1),axis=0)
a.plot(kind="bar",stacked=True)

### 料金毎

In [ ]:
ranges =[-1,10,20,30,40,520]
merge_df["Fares"]=pd.cut(merge_df["Fare"],ranges)

In [ ]:
a=merge_df.sort_values(by="Fares").pivot_table("C",aggfunc="count",index="Fares",columns="Survived")

a.plot(kind="bar",stacked=True)